In [1]:
import os
import json
import pandas as pd
import traceback


In [2]:
from langchain_openai import ChatOpenAI


In [6]:
import os
key=os.getenv('OPENAI_API_KEY')

In [7]:
from dotenv import load_dotenv
load_dotenv()



True

In [12]:
llm = ChatOpenAI(
    openai_api_key=OPEN_API_KEY,  # ✅ correct argument name
    model="gpt-4o-mini",
    temperature=0.5
)

In [13]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001AB8485BA10>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001AB85D90590>, root_client=<openai.OpenAI object at 0x000001AB85A382D0>, root_async_client=<openai.AsyncOpenAI object at 0x000001AB85D902F0>, model_name='gpt-4o-mini', temperature=0.5, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [14]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [15]:
TEMPLATE="""
Text:{text}

you are an expert MCqs maker, given the above text. It is your job to create a quiz of {numbers},
 multiple quiz-wise questions for {subject}, students in {tone} tone. In curly brackets,
   make sure the questions are not repeated and check all the questions to be confirming the text as well.
     Make sure to format your response like response.json below and use it as a guide. Make sure to make dash {numbers} of MCqs
     
# response.json.
{response_json}"""

In [16]:
quiz_generation_prompt=PromptTemplate(
    input_variable=['text','number','subject','tone','response.json'],
    template=TEMPLATE
)

In [17]:
RESPONSE_JSON={
    "1":{
        'mcq':'multiple choice question',
        'option': {
            'a':'choice here',
            'b' :'choice here',
            'c':'choice here',
            'd':  'choice here'            
    },
    'correct':'correct answer',
    },
   "2":{
        'mcq':'multiple choice question',
        'option': {
            'a':'choice here',
            'b' :'choice here',
            'c':'choice here',
            'd':  'choice here'            
    },
    'correct':'correct answer'
    },
"3":{
        'mcq':'multiple choice question',
        'option': {
            'a':'choice here',
            'b' :'choice here',
            'c':'choice here',
            'd':  'choice here'            
    },
    'correct':'correct answer',
    },
    "4":{
        'mcq':'multiple choice question',
        'option': {
            'a':'choice here',
            'b' :'choice here',
            'c':'choice here',
            'd':  'choice here'            
    },
    'correct':'correct answer',
    },

}




In [18]:
# chain
quiz_chain=LLMChain(llm=llm,
                    prompt=quiz_generation_prompt,
                    output_key='quiz',verbose=True)

C:\Users\user\AppData\Local\Temp\ipykernel_5216\862413768.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm,


In [19]:
TEMPLATE2="""
You are an expert teacher .given a multiple choice quiz for {subject} students.
You need to evalute the complexity of the question and give a complete analysis of the quiz .only use 50 words for complexity 
if the quiz is not per with the congitive and analytical abilities of the student.
Update the quiz questions which needs to be changed and change the tone such that is perfectly fits the student level.
Check the quiz answer realted to the content .
Quiz_MCQS
{quiz}

"""

In [20]:
quiz_generation_prompt=PromptTemplate(
    input_variable=['subject','quiz'],
    template=TEMPLATE2
)

In [21]:
# chain2
review_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,
                      output_key='review',
                      verbose=True
                      )

In [22]:

generate_chains = SequentialChain(
    chains=[quiz_chain, review_chain], 
    input_variables=['text', 'numbers', 'subject', 'tone', 'response_json'],  # ✅ cleaned
    output_variables=["quiz", "review"],                                     # ✅ matches invoke
    verbose=True
)





In [23]:
file_path = r"C:\Users\user\mcqgen\experiment\data.txt"  # raw string prevents \ issues
with open(file_path, 'r', encoding='utf-8') as file:     # encoding avoids Unicode errors
    text = file.read()

 # preview first 200 characters


In [24]:
print(text)


Artificial intelligence (AI) is technology that enables computers and machines to simulate human learning, comprehension, problem solving, decision making, creativity and autonomy.

Applications and devices equipped with AI can see and identify objects. They can understand and respond to human language. They can learn from new information and experience. They can make detailed recommendations to users and experts. They can act independently, replacing the need for human intelligence or intervention (a classic example being a self-driving car).

But in 2024, most AI researchers, practitioners and most AI-related headlines are focused on breakthroughs in generative AI (gen AI), a technology that can create original text, images, video and other content. To fully understand generative AI, it’s important to first understand the technologies on which generative AI tools are built: machine learning (ML) and deep learningDirectly underneath AI, we have machine learning, which involves creati

In [25]:
#convert python dict into json formatted
quiz=json.dumps(RESPONSE_JSON)

In [26]:
quiz

'{"1": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [27]:
# Define chain inputs
numbers = 5                # how many MCQs you want (use singular, not numbers)
subject = "Artifical Intelligence"        # or any subject you need
tone = "student-friendly"  # style of the quiz
RESPONSE_JSON = {
    "quiz": [],
    "review": ""
}


In [28]:
with get_openai_callback() as cb:
    response = generate_chains.invoke(
        {
            "text": text,             # ✅ matches chain input
            "numbers": numbers,                  # ✅ singular, not "numbers"
            "subject": subject,
            "tone": tone,
            "response_json": json.dumps(RESPONSE_JSON),
           

        }
    )





> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:
Artificial intelligence (AI) is technology that enables computers and machines to simulate human learning, comprehension, problem solving, decision making, creativity and autonomy.

Applications and devices equipped with AI can see and identify objects. They can understand and respond to human language. They can learn from new information and experience. They can make detailed recommendations to users and experts. They can act independently, replacing the need for human intelligence or intervention (a classic example being a self-driving car).

But in 2024, most AI researchers, practitioners and most AI-related headlines are focused on breakthroughs in generative AI (gen AI), a technology that can create original text, images, video and other content. To fully understand generative AI, it’s important to first understand the technologies on which generative AI tools are built: ma

In [29]:
print("Quiz:", response["quiz"])

print("Review:", response["review"])
print("Tokens used:", cb.total_tokens)
print("total cost:",cb.total_cost)
print("prompt Token:",cb.prompt_tokens)

Quiz: ```json
{
  "quiz": [
    {
      "question": "What does artificial intelligence (AI) enable computers and machines to do?",
      "options": [
        "A) Simulate human emotions",
        "B) Simulate human learning and decision making",
        "C) Replace all human jobs",
        "D) Only perform mathematical calculations"
      ],
      "answer": "B"
    },
    {
      "question": "Which technology is primarily focused on creating original content like text and images?",
      "options": [
        "A) Machine Learning",
        "B) Deep Learning",
        "C) Generative AI",
        "D) Supervised Learning"
      ],
      "answer": "C"
    },
    {
      "question": "What is the simplest form of machine learning that uses labeled data sets?",
      "options": [
        "A) Unsupervised Learning",
        "B) Reinforcement Learning",
        "C) Supervised Learning",
        "D) Generative Learning"
      ],
      "answer": "C"
    },
    {
      "question": "What type of mac

In [41]:
quiz


'```json\n{\n  "quiz": [\n    {\n      "question": "What does artificial intelligence (AI) enable computers and machines to do?",\n      "options": [\n        "A) Simulate human emotions",\n        "B) Simulate human learning and decision making",\n        "C) Replace all human jobs",\n        "D) Only perform mathematical calculations"\n      ],\n      "answer": "B"\n    },\n    {\n      "question": "Which technology is primarily focused on creating original content like text and images?",\n      "options": [\n        "A) Machine Learning",\n        "B) Deep Learning",\n        "C) Generative AI",\n        "D) Supervised Learning"\n      ],\n      "answer": "C"\n    },\n    {\n      "question": "What is the simplest form of machine learning that uses labeled data sets?",\n      "options": [\n        "A) Unsupervised Learning",\n        "B) Reinforcement Learning",\n        "C) Supervised Learning",\n        "D) Generative Learning"\n      ],\n      "answer": "C"\n    },\n    {\n      

In [30]:
quiz = json.dumps(RESPONSE_JSON)


In [42]:
quiz=response["quiz"]

In [48]:
import json
import re

# Remove any text before the first { and after the last }
json_match = re.search(r"\{.*\}", quiz, re.DOTALL)
if not json_match:
    raise ValueError("No valid JSON found in the quiz output!")

clean_quiz = json_match.group(0)

# Now parse
quiz_data = json.loads(clean_quiz)

# Build quiz table
quiz_table_data = []
for value in quiz_data.get("quiz", []):
    mcq = value.get("question", "No question provided")
    options = value.get("options", [])
    options_str = " | ".join(options) if options else "No options provided"
    correct = value.get("answer", "No answer provided")
    
    quiz_table_data.append({
        "MCQ": mcq,
        "Choices": options_str,
        "Correct": correct
    })

quiz_table_data



[{'MCQ': 'What does artificial intelligence (AI) enable computers and machines to do?',
  'Choices': 'A) Simulate human emotions | B) Simulate human learning and decision making | C) Replace all human jobs | D) Only perform mathematical calculations',
  'Correct': 'B'},
 {'MCQ': 'Which technology is primarily focused on creating original content like text and images?',
  'Choices': 'A) Machine Learning | B) Deep Learning | C) Generative AI | D) Supervised Learning',
  'Correct': 'C'},
 {'MCQ': 'What is the simplest form of machine learning that uses labeled data sets?',
  'Choices': 'A) Unsupervised Learning | B) Reinforcement Learning | C) Supervised Learning | D) Generative Learning',
  'Correct': 'C'},
 {'MCQ': 'What type of machine learning algorithm is modeled after the human brain?',
  'Choices': 'A) Decision Trees | B) Neural Networks | C) Random Forests | D) Support Vector Machines',
  'Correct': 'B'},
 {'MCQ': 'What is the primary goal of machine learning algorithms in supervi

In [46]:
import json

# Check if quiz is not empty
if not quiz or not quiz.strip():
    raise ValueError("Quiz data is empty. Cannot parse JSON.")

# Try parsing JSON safely
try:
    quiz_data = json.loads(quiz)  # Convert string → dict
except json.JSONDecodeError as e:
    raise ValueError(f"Invalid JSON in quiz data: {e}")

quiz_table_data = []

# Loop through each question safely
for value in quiz_data.get("quiz", []):
    mcq = value.get("question", "No question provided")
    options = value.get("options", [])
    options_str = " | ".join(options) if options else "No options provided"
    correct = value.get("answer", "No answer provided")
    
    quiz_table_data.append({
        "MCQ": mcq,
        "Choices": options_str,
        "Correct": correct
    })

quiz_table_data





ValueError: Invalid JSON in quiz data: Expecting value: line 1 column 1 (char 0)

In [44]:
print(repr(quiz))  # Shows exactly what's inside, even if it's empty


'```json\n{\n  "quiz": [\n    {\n      "question": "What does artificial intelligence (AI) enable computers and machines to do?",\n      "options": [\n        "A) Simulate human emotions",\n        "B) Simulate human learning and decision making",\n        "C) Replace all human jobs",\n        "D) Only perform mathematical calculations"\n      ],\n      "answer": "B"\n    },\n    {\n      "question": "Which technology is primarily focused on creating original content like text and images?",\n      "options": [\n        "A) Machine Learning",\n        "B) Deep Learning",\n        "C) Generative AI",\n        "D) Supervised Learning"\n      ],\n      "answer": "C"\n    },\n    {\n      "question": "What is the simplest form of machine learning that uses labeled data sets?",\n      "options": [\n        "A) Unsupervised Learning",\n        "B) Reinforcement Learning",\n        "C) Supervised Learning",\n        "D) Generative Learning"\n      ],\n      "answer": "C"\n    },\n    {\n      

In [34]:
quiz

'```json\n{\n  "quiz": [\n    {\n      "question": "What does artificial intelligence (AI) enable computers and machines to do?",\n      "options": [\n        "A) Simulate human emotions",\n        "B) Simulate human learning and decision making",\n        "C) Replace all human jobs",\n        "D) Only perform mathematical calculations"\n      ],\n      "answer": "B"\n    },\n    {\n      "question": "Which technology is primarily focused on creating original content like text and images?",\n      "options": [\n        "A) Machine Learning",\n        "B) Deep Learning",\n        "C) Generative AI",\n        "D) Supervised Learning"\n      ],\n      "answer": "C"\n    },\n    {\n      "question": "What is the simplest form of machine learning that uses labeled data sets?",\n      "options": [\n        "A) Unsupervised Learning",\n        "B) Reinforcement Learning",\n        "C) Supervised Learning",\n        "D) Generative Learning"\n      ],\n      "answer": "C"\n    },\n    {\n      

In [35]:
print(response)


{'text': "\nArtificial intelligence (AI) is technology that enables computers and machines to simulate human learning, comprehension, problem solving, decision making, creativity and autonomy.\n\nApplications and devices equipped with AI can see and identify objects. They can understand and respond to human language. They can learn from new information and experience. They can make detailed recommendations to users and experts. They can act independently, replacing the need for human intelligence or intervention (a classic example being a self-driving car).\n\nBut in 2024, most AI researchers, practitioners and most AI-related headlines are focused on breakthroughs in generative AI (gen AI), a technology that can create original text, images, video and other content. To fully understand generative AI, it’s important to first understand the technologies on which generative AI tools are built: machine learning (ML) and deep learningDirectly underneath AI, we have machine learning, which 

In [36]:

file = quiz_df.to_csv("artificial_intelligence.csv", index=False)


NameError: name 'quiz_df' is not defined

In [50]:
import json
import pandas as pd
import re

# Check raw output
print("Raw quiz output:", repr(response.get("quiz", "")))

quiz_raw = response.get("quiz", "")
if not quiz_raw.strip():
    raise ValueError("Quiz data is empty! Check your chain output.")

# Extract JSON object from the string (in case there's extra text)
match = re.search(r"\{.*\}", quiz_raw, re.DOTALL)
if not match:
    raise ValueError("No valid JSON found in the quiz output!")

quiz_clean = match.group(0)

# Parse JSON
quiz_dict = json.loads(quiz_clean)

# Extract quiz questions safely
quiz_list = quiz_dict.get("quiz", [])

# Transform for CSV
quiz_table = []
for q in quiz_list:
    mcq = q.get("question", "No question provided")
    options = q.get("options", [])
    options_str = " | ".join(options) if options else "No options"
    answer = q.get("answer", "No answer")
    
    quiz_table.append({
        "MCQ": mcq,
        "Choices": options_str,
        "Correct": answer
    })

# Convert to DataFrame and save CSV
df = pd.DataFrame(quiz_table)
df.to_csv("artificial_intelligence.csv", index=False)
print(f"✅ Saved CSV with {len(df)} rows")


Raw quiz output: '```json\n{\n  "quiz": [\n    {\n      "question": "What does artificial intelligence (AI) enable computers and machines to do?",\n      "options": [\n        "A) Simulate human emotions",\n        "B) Simulate human learning and decision making",\n        "C) Replace all human jobs",\n        "D) Only perform mathematical calculations"\n      ],\n      "answer": "B"\n    },\n    {\n      "question": "Which technology is primarily focused on creating original content like text and images?",\n      "options": [\n        "A) Machine Learning",\n        "B) Deep Learning",\n        "C) Generative AI",\n        "D) Supervised Learning"\n      ],\n      "answer": "C"\n    },\n    {\n      "question": "What is the simplest form of machine learning that uses labeled data sets?",\n      "options": [\n        "A) Unsupervised Learning",\n        "B) Reinforcement Learning",\n        "C) Supervised Learning",\n        "D) Generative Learning"\n      ],\n      "answer": "C"\n    

In [51]:

quiz_table

[{'MCQ': 'What does artificial intelligence (AI) enable computers and machines to do?',
  'Choices': 'A) Simulate human emotions | B) Simulate human learning and decision making | C) Replace all human jobs | D) Only perform mathematical calculations',
  'Correct': 'B'},
 {'MCQ': 'Which technology is primarily focused on creating original content like text and images?',
  'Choices': 'A) Machine Learning | B) Deep Learning | C) Generative AI | D) Supervised Learning',
  'Correct': 'C'},
 {'MCQ': 'What is the simplest form of machine learning that uses labeled data sets?',
  'Choices': 'A) Unsupervised Learning | B) Reinforcement Learning | C) Supervised Learning | D) Generative Learning',
  'Correct': 'C'},
 {'MCQ': 'What type of machine learning algorithm is modeled after the human brain?',
  'Choices': 'A) Decision Trees | B) Neural Networks | C) Random Forests | D) Support Vector Machines',
  'Correct': 'B'},
 {'MCQ': 'What is the primary goal of machine learning algorithms in supervi

In [52]:
quiz_data


{'quiz': [{'question': 'What does artificial intelligence (AI) enable computers and machines to do?',
   'options': ['A) Simulate human emotions',
    'B) Simulate human learning and decision making',
    'C) Replace all human jobs',
    'D) Only perform mathematical calculations'],
   'answer': 'B'},
  {'question': 'Which technology is primarily focused on creating original content like text and images?',
   'options': ['A) Machine Learning',
    'B) Deep Learning',
    'C) Generative AI',
    'D) Supervised Learning'],
   'answer': 'C'},
  {'question': 'What is the simplest form of machine learning that uses labeled data sets?',
   'options': ['A) Unsupervised Learning',
    'B) Reinforcement Learning',
    'C) Supervised Learning',
    'D) Generative Learning'],
   'answer': 'C'},
  {'question': 'What type of machine learning algorithm is modeled after the human brain?',
   'options': ['A) Decision Trees',
    'B) Neural Networks',
    'C) Random Forests',
    'D) Support Vector Mac